# Convolutional Neural Networks with Keras (2)

#### Alternatives when using a pre-trained network:

- Just use softmax predictions for the new data. Only makes sense when new data have known classes.
- Use the bottleneck features (activations from last MaxPooling layer before fully connected layers)


Steps:

1) instantiate convolutional part of the model (everything up to the fully-connected layers) and run this model on the training and test data once, recording the output (the "bottleneck features", i.e. the last activation maps before the fully-connected layers) in two numpy arrays

2) train a small fully-connected model on top of the stored features

In [23]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import numpy as np
import os

In [3]:
img_width, img_height = 375,500

train_data_dir = 'data/train'
test_data_dir = 'data/test'

n_train_samples = 235
n_train_ants = 114
n_train_bees = 121


n_test_samples = 148
n_test_ants = 66
n_test_bees = 82

num_epochs = 50
batch_size = 16

top_model_weights_path = 'bottleneck_fc_model.h5'
bottleneck_features_train_path = "bottleneck_features_train.npy"
bottleneck_features_test_path = "bottleneck_features_test.npy"

### Step 1: save bottleneck features

The reason why we are storing the features offline rather than adding our fully-connected model directly on top of a frozen convolutional base and running the whole thing, is computational effiency.

In [4]:
# no data augmentation
datagen = ImageDataGenerator(rescale=1. / 255)

# training data
generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, # no labels
        shuffle=False)

Found 235 images belonging to 2 classes.


In [5]:
# load the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

In [6]:
bottleneck_features_train = model.predict_generator(generator, n_train_samples//batch_size+1, verbose=1)

15/15 [==============================] - 734s    


In [7]:
bottleneck_features_train.shape

(235, 11, 15, 512)

In [9]:
np.save(bottleneck_features_train_path, bottleneck_features_train)

In [10]:
# test data
generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

bottleneck_features_test = model.predict_generator(generator,n_test_samples // batch_size + 1, verbose=1)


Found 148 images belonging to 2 classes.
10/10 [==============================] - 469s    


In [12]:
bottleneck_features_test.shape

(148, 11, 15, 512)

In [13]:
np.save(bottleneck_features_test_path, bottleneck_features_test)

### Step 2: load saved data and train a small fully-connected model on top

In [14]:
train_data = np.load(bottleneck_features_train_path)
train_labels = np.array([0] * n_train_ants + [1] * n_train_bees)
train_labels.shape, train_data.shape

((235,), (235, 11, 15, 512))

In [15]:
test_data = np.load(bottleneck_features_test_path)
test_labels = np.array([0] * n_test_ants + [1] * n_test_bees)
test_labels.shape, test_data.shape

((148,), (148, 11, 15, 512))

In [16]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 84480)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               21627136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 21,627,393
Trainable params: 21,627,393
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels,
              epochs=num_epochs,
              batch_size=batch_size,
              validation_data=(test_data, test_labels))
model.save_weights(top_model_weights_path)


Train on 235 samples, validate on 148 samples
Epoch 1/50
235/235 [==============================] - 4s - loss: 7.3266 - acc: 0.5106 - val_loss: 7.1094 - val_acc: 0.5541
Epoch 2/50
235/235 [==============================] - 4s - loss: 7.7338 - acc: 0.5149 - val_loss: 7.1094 - val_acc: 0.5541
Epoch 3/50
235/235 [==============================] - 4s - loss: 7.7338 - acc: 0.5149 - val_loss: 7.1094 - val_acc: 0.5541
Epoch 4/50
235/235 [==============================] - 4s - loss: 7.7338 - acc: 0.5149 - val_loss: 7.1094 - val_acc: 0.5541
Epoch 5/50
235/235 [==============================] - 4s - loss: 7.7338 - acc: 0.5149 - val_loss: 7.1094 - val_acc: 0.5541
Epoch 6/50
235/235 [==============================] - 4s - loss: 7.7270 - acc: 0.5149 - val_loss: 7.1094 - val_acc: 0.5541
Epoch 7/50
235/235 [==============================] - 4s - loss: 7.0320 - acc: 0.5277 - val_loss: 8.9303 - val_acc: 0.4459
Epoch 8/50
235/235 [==============================] - 4s - loss: 6.3234 - acc: 0.5532 - val_l

### Test the model

At this time, there is no overall model that could be used for new predictions.

This will be addressed in the next notebook.